In [1]:
import ee
import geemap 

ee.Initialize()

In [2]:
class S2Collection:
    def __init__(self, start_date, end_date, aoi) -> None:
        self.start_date = start_date
        self.end_date = end_date
        self.aoi = aoi
        self.dataset = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
    
    def cloud_mask(self):
        def maskS2clouds(image):
            qa = image.select('QA60')
            cloudBitMask = 1 << 10
            cirrusBitMask = 1 << 11
            mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
                qa.bitwiseAnd(cirrusBitMask).eq(0))
            return image.updateMask(mask).divide(10000)
        self.dataset = self.dataset.map(maskS2clouds)
        return self
    
    def __add__(self, other):
        self.dataset = self.dataset.merge(other.dataset)
        return self
    
    def process(self):
        self.dataset = self.dataset.filterDate(self.start_date, self.end_date)
        self.dataset = self.dataset.filterBounds(self.aoi)
        self.dataset = self.dataset.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        self.cloud_mask()
        return self
        

In [3]:
aoi = ee.FeatureCollection("projects/yk-wetland-mapping/assets/yukon-bounds")

s2_2018 = S2Collection('2018-06-20', '2018-09-21', aoi).process()
s2_2019 = S2Collection('2019-06-20', '2019-09-21', aoi).process()
s2_2020 = S2Collection('2020-06-20', '2020-09-21', aoi).process()

s2_combined = s2_2018 + s2_2019 + s2_2020

s2_composite = s2_combined.dataset.median()

In [4]:
Map = geemap.Map()
vis = {'bands': ['B4', 'B3', 'B2'],'min': 0.0, 'max': 0.3}
Map.addLayer(s2_composite, vis, f'S2 (B4, B3, B2): 2018-2020')
Map.addLayerControl()

Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…